In [ ]:
# Install the pygeohash library, which allows encoding and decoding of geohashes (used for spatial indexing and grouping)
!pip install pygeohash

In [ ]:
# Import Required Libraries
# os is used for handling file paths and directory operations
import os
# pandas is used for data manipulation and analysis using DataFrames
import pandas as pd
# numpy provides support for numerical operations and array handling
import numpy as np
# geopandas extends pandas to handle geospatial data using shapefiles, geometry columns, etc.
import geopandas as gpd
# DBSCAN is a clustering algorithm from scikit-learn used for density-based clustering
from sklearn.cluster import KMeans, DBSCAN
# MinMaxScaler is used to scale features to a specified range, typically [0, 1]
from sklearn.preprocessing import MinMaxScaler
# mean_absolute_error is used to evaluate prediction performance by calculating average absolute errors
from sklearn.metrics import mean_absolute_error
# great_circle is used to calculate the shortest distance between two points on the Earth's surface
from geopy.distance import great_circle
# pygeohash is used for encoding latitude and longitude into geohash strings for spatial grouping
import pygeohash as pgh
# folium is used for interactive map visualization in Python
import folium
# MarkerCluster allows grouping markers in folium maps for better readability
from folium.plugins import MarkerCluster
# matplotlib is a standard plotting library for creating static, animated, and interactive visualizations
import matplotlib.pyplot as plt
# seaborn is a statistical data visualization library based on matplotlib, providing enhanced plots
import seaborn as sns
# shapely.geometry.Point is used to create point objects for spatial operations
from shapely.geometry import Point
# nearest_points is used to find the nearest point between two geometries
from shapely.ops import nearest_points
# google.colab.drive is used to mount Google Drive in Colab to access files stored there
from google.colab import drive
# Calculates the Silhouette Score, which is used to evaluate the quality of clustering results. A higher score indicates better-defined clusters.
from sklearn.metrics import silhouette_score
# Splits the dataset into training and testing subsets. This is typically used in machine learning to evaluate model performance by training on one set and testing on another.
from sklearn.model_selection import train_test_split
# Import folium for interactive map visualization of spatial data
import folium

In [ ]:
# Step 1: Mount Drive & Load AQ and Taxi Data
# Mount Google Drive to access files stored in the Drive
drive.mount('/content/drive')
# Set the path to the directory where the datasets are stored
base_path = '/content/drive/MyDrive/Datasets/'

# Initialize an empty list to collect data from each AQ CSV file
aq_dfs = []
# Loop through AQ_data_1.csv to AQ_data_19.csv to read each file
for i in range(1, 20):
    # Construct the file path for each AQ dataset
    file_path = os.path.join(base_path, f"AQ_data_{i}.csv")

    # Check if the file exists at the specified path
    if os.path.exists(file_path):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Rename the necessary columns for standardization
        df = df.rename(columns={
            'Latitude': 'latitude',
            'Longitude': 'longitude',
            'ReadingDateTimeUTC': 'timestamp',
            'PM25': 'pm25'
        })

        # Keep only the relevant columns needed for analysis
        df = df[['latitude', 'longitude', 'timestamp', 'pm25']]

        # Print how many records were loaded from the current file
        print(f" Loaded AQ_data_{i}.csv with {len(df)} records")

        # Append the DataFrame to the AQ list
        aq_dfs.append(df)
    else:
        # If the file doesn't exist, print a message
        print(f" File not found: {file_path}")

# Combine all individual AQ DataFrames into a single DataFrame
aq_df = pd.concat(aq_dfs, ignore_index=True)

# Display a preview of the first few records of the combined AQ data
print(" Preview of Combined AQ Data:")
print(aq_df.head())

# Print the total number of AQ records loaded
print(f" Total AQ Records Loaded: {len(aq_df)}")

# Load All Taxi Trips Data
# Construct the full file path to the Taxi_Trips dataset
taxi_file_path = os.path.join(base_path, 'Taxi_Trips.csv')

# Load selected columns from the taxi dataset for memory efficiency
taxi_df = pd.read_csv(taxi_file_path, usecols=[
    'Trip Start Timestamp',
    'Trip Seconds',
    'Trip Miles',
    'Pickup Centroid Latitude',
    'Pickup Centroid Longitude'
])

# Rename the columns for consistency with other datasets
taxi_df = taxi_df.rename(columns={
    'Trip Start Timestamp': 'trip_start_timestamp',
    'Pickup Centroid Latitude': 'pickup_latitude',
    'Pickup Centroid Longitude': 'pickup_longitude',
    'Trip Seconds': 'trip_seconds',
    'Trip Miles': 'trip_miles'
})

# Display a preview of the taxi data
print("\n Preview of Taxi Trips Data:")
print(taxi_df.head())

# Print the total number of taxi trip records loaded
print(f" Total Taxi Records Loaded: {len(taxi_df)}")

In [ ]:
# Step 2: Convert Timestamps & Resample by 5-Hour Windows
# Convert AQ 'timestamp' column from string format to pandas datetime format
aq_df['timestamp'] = pd.to_datetime(aq_df['timestamp'])

# Print a few converted AQ timestamps to verify the format
print("AQ timestamps converted:")
print(aq_df[['timestamp']].head())

# Convert Taxi 'trip_start_timestamp' column from string to datetime format
taxi_df['trip_start_timestamp'] = pd.to_datetime(taxi_df['trip_start_timestamp'])

# Print a few converted Taxi timestamps to verify the format
print("\nTaxi trip timestamps converted:")
print(taxi_df[['trip_start_timestamp']].head())

# Round each AQ timestamp down to the nearest 5-hour interval (e.g., 00:00, 05:00, 10:00...)
aq_df['time_block'] = aq_df['timestamp'].dt.floor('5H')

# Show the original and new 5-hour rounded timestamps for AQ data
print("\nAQ data after assigning to 5-hour time blocks:")
print(aq_df[['timestamp', 'time_block']].head())

# Round each Taxi trip timestamp to the nearest 5-hour block
taxi_df['time_block'] = taxi_df['trip_start_timestamp'].dt.floor('5H')

# Show original and new 5-hour rounded timestamps for Taxi data
print("\nTaxi data after assigning to 5-hour time blocks:")
print(taxi_df[['trip_start_timestamp', 'time_block']].head())

In [ ]:
# Step 3: Geohash Grouping and Feature Aggregation
# Define a helper function to compute geohashes based on latitude and longitude
def geohash_group(df, lat_col, lon_col, precision=5):
    """
    Adds a 'geohash' column to the DataFrame based on latitude and longitude.

    Args:
        df (pd.DataFrame): Input DataFrame
        lat_col (str): Name of the latitude column
        lon_col (str): Name of the longitude column
        precision (int): Geohash precision level (higher = smaller area)

    Returns:
        pd.DataFrame: DataFrame with 'geohash' column added
    """
    # Apply geohash encoding to each row using specified lat/lon columns
    df['geohash'] = df.apply(lambda row: pgh.encode(row[lat_col], row[lon_col], precision=precision), axis=1)
    return df

# Apply geohash encoding to AQ data
aq_df = geohash_group(aq_df, 'latitude', 'longitude')

# Apply geohash encoding to Taxi data
taxi_df = geohash_group(taxi_df, 'pickup_latitude', 'pickup_longitude')

# Print a sample of AQ geohash assignments
print(" Sample AQ geohashes:")
print(aq_df[['latitude', 'longitude', 'geohash']].head())

# Print a sample of Taxi geohash assignments
print("\n Sample Taxi geohashes:")
print(taxi_df[['pickup_latitude', 'pickup_longitude', 'geohash']].head())

# Aggregate taxi data: count number of trips per geohash and time block (trip density)
taxi_density = taxi_df.groupby(['geohash', 'time_block']).size().reset_index(name='taxi_density')
print("\n Taxi density (trips per geohash and time block):")
print(taxi_density.head())

# Aggregate AQ data: calculate average PM2.5 per geohash and time block
aq_avg = aq_df.groupby(['geohash', 'time_block'])['pm25'].mean().reset_index(name='pm25')
print("\n Average PM2.5 per geohash and time block:")
print(aq_avg.head())

# Merge the two datasets on common geohash and time block to align spatial-temporal features
features_df = pd.merge(taxi_density, aq_avg, on=['geohash', 'time_block'])

# Print the merged features combining taxi activity and air quality
print("\n Merged Features DataFrame (Taxi + AQ):")
print(features_df.head())

# Show total number of records in the final dataset
print(f" Total Records in Feature Set: {len(features_df)}")

In [ ]:
# Step 4: Compute Weighted Feature for Clustering

# Define the weight (lambda) that determines the importance of air quality in the combined feature
lambda_ = 0.6  # Weight for air quality influence (60% AQ, 40% taxi density)

# Create a new weighted feature that blends PM2.5 air quality and taxi density
# This will be used as a combined metric for clustering
features_df['weighted_feature'] = lambda_ * features_df['pm25'] + (1 - lambda_) * features_df['taxi_density']

# Decode the geohash back into latitude and longitude coordinates
# This step is useful for visualizing or mapping the clusters later
features_df[['latitude', 'longitude']] = features_df['geohash'].apply(lambda g: pd.Series(pgh.decode(g)))

In [ ]:
# Step 5: Normalize and Cluster Using DBSCAN and KMeans + Evaluate
# Step 5.1: Select features for clustering
# We're clustering based on longitude, latitude, and a weighted feature (e.g., combining taxi density + PM2.5)
X = features_df[['longitude', 'latitude', 'weighted_feature']].values

# Step 5.2: Normalize the features to a [0,1] range using Min-Max Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Step 5.3: Apply DBSCAN clustering
# DBSCAN is good for finding arbitrarily shaped clusters and dealing with noise
dbscan = DBSCAN(eps=0.05, min_samples=5)
features_df['dbscan_cluster'] = dbscan.fit_predict(X_scaled)

# Step 5.4: Apply KMeans clustering for comparison
# KMeans assumes spherical clusters and needs a fixed number of clusters (n_clusters)
kmeans = KMeans(n_clusters=8, random_state=42, n_init='auto')  # You can tune n_clusters
features_df['kmeans_cluster'] = kmeans.fit_predict(X_scaled)

# Step 5.5: Evaluate clustering using Silhouette Score
# Silhouette Score ranges from -1 to 1; higher is better
# DBSCAN: Ignore noise points labeled as -1
dbscan_mask = features_df['dbscan_cluster'] != -1
if dbscan_mask.sum() > 1:
    dbscan_score = silhouette_score(X_scaled[dbscan_mask], features_df['dbscan_cluster'][dbscan_mask])
else:
    dbscan_score = -1  # Not enough clusters for scoring

# KMeans: Score is calculated on all points
kmeans_score = silhouette_score(X_scaled, features_df['kmeans_cluster'])

print(f" Silhouette Score - DBSCAN: {dbscan_score:.4f}")
print(f" Silhouette Score - KMeans: {kmeans_score:.4f}")

# Step 5.6: Stratified Sampling Based on Geohash
# This helps ensure train/test split maintains similar spatial distribution
train_df, test_df = train_test_split(
    features_df, test_size=0.3, stratify=features_df['geohash'], random_state=42
)
print(f" Train size: {len(train_df)}, Test size: {len(test_df)}")

In [ ]:
# Step 6: Determine Sensor Candidate Locations (Centroids)
# Group the data by DBSCAN cluster labels and compute the mean longitude, latitude, and PM2.5 for each cluster.
# These means represent the centroids—ideal candidate locations for placing sensors.
cluster_centroids = features_df.groupby('dbscan_cluster')[['longitude', 'latitude', 'pm25']].mean().reset_index()

# Exclude noise points labeled as -1 by DBSCAN, as they don't belong to any meaningful cluster.
cluster_centroids = cluster_centroids[cluster_centroids['dbscan_cluster'] != -1]

# Save the resulting centroids to a CSV file for further analysis or deployment.
cluster_centroids.to_csv('sensor_candidates.csv', index=False)

In [ ]:
# Step 7: Accuracy Evaluation 
# This function evaluates the accuracy of the sensor candidate locations by comparing their PM2.5 values
# to the nearest actual ground truth air quality readings (from aq_df).
def evaluate_sensor_accuracy(cluster_centers, ground_truth_df):
    errors = []
    for _, center in cluster_centers.iterrows():
        center_point = (center['latitude'], center['longitude'])

        # For each cluster center, compute the distance to all ground truth points
        ground_truth_df['distance'] = ground_truth_df.apply(
            lambda row: great_circle(center_point, (row['latitude'], row['longitude'])).meters,
            axis=1
        )

        # Find the nearest ground truth point to the cluster center
        nearest = ground_truth_df.loc[ground_truth_df['distance'].idxmin()]

        # Compare PM2.5 at the center with the nearest ground truth PM2.5 value
        predicted_pm25 = center['pm25']
        true_pm25 = nearest['pm25']
        errors.append(abs(predicted_pm25 - true_pm25))  # Store the absolute error

    return np.mean(errors)  # Return the Mean Absolute Error (MAE)

# Evaluate the sensor candidate locations and print the MAE
mae = evaluate_sensor_accuracy(cluster_centroids, aq_df)
print(f"Mean Absolute Error of sensor placement: {mae:.2f}")

In [ ]:
# Step 8: Enrich with External Datasets
# Convert cluster_centroids to GeoDataFrame
gdf_clusters = gpd.GeoDataFrame(
    cluster_centroids,
    geometry=[Point(xy) for xy in zip(cluster_centroids.longitude, cluster_centroids.latitude)],
    crs="EPSG:4326"
)

# Load external geospatial layers 
base_path = '/content/drive/MyDrive/Datasets/' 

gdf_bus_stops = gpd.read_file(os.path.join(base_path, "CTA_BusStops.shp"))
# Urban_Farms data is in a CSV and needs to be converted to a GeoDataFrame:
urban_farms_df = pd.read_csv(os.path.join(base_path, "Urban_Farms.csv")) # Read as DataFrame using pandas
# If 'Urban_Farms.csv' has geometry info, create a GeoDataFrame:
gdf_urban_farms = gpd.GeoDataFrame(urban_farms_df, geometry=gpd.points_from_xy(urban_farms_df.LONGITUDE, urban_farms_df.LATITUDE), crs="EPSG:4326")


# Load Rail Lines data
rail_lines_df = pd.read_csv(os.path.join(base_path, "Rail_Lines.csv"))
# Convert 'the_geom' column to Shapely geometry objects
rail_lines_df['geometry'] = gpd.GeoSeries.from_wkt(rail_lines_df['the_geom'])
# Create GeoDataFrame using the converted geometry column
gdf_rail_lines = gpd.GeoDataFrame(rail_lines_df, geometry='geometry', crs="EPSG:4326")


#  Alt_Fuel_Stations.csv has latitude and longitude columns
alt_fuel_df = pd.read_csv(os.path.join(base_path, "Alt_Fuel_Stations.csv")) 
# Create a GeoDataFrame from the DataFrame
gdf_alt_fuel = gpd.GeoDataFrame(alt_fuel_df, geometry=gpd.points_from_xy(alt_fuel_df.Longitude, alt_fuel_df.Latitude), crs="EPSG:4326") # Assuming 'Longitude' and 'Latitude' are column names


# 'Census_Tracts.csv' has geometry information in 'the_geom' column
# If it has latitude and longitude, replace 'the_geom' with gpd.points_from_xy(census_df.longitude, census_df.latitude)
# Load the census data first as a regular DataFrame
census_df = pd.read_csv(os.path.join(base_path, "Census_Tracts.csv"))
gdf_census = gpd.GeoDataFrame(census_df, geometry=gpd.GeoSeries.from_wkt(census_df['the_geom']), crs="EPSG:4326") # Use census_df here


df_wind = pd.read_csv(os.path.join(base_path, "Beach_Weather_Stations.csv"))
# Add coordinates by station name
station_coords = {
    "63rd Street Beach": (41.780992, -87.572619),
    "Foster Beach": (41.976816, -87.651546),
    "Montrose Beach": (41.961708, -87.638719),
    "Oak Street Beach": (41.910337, -87.626123),
    "Ohio Street Beach": (41.894000, -87.613000),
    "Rainbow Beach": (41.761000, -87.565000),
    "North Avenue Beach": (41.911000, -87.628000)
}

def get_coords(station_name):
    return pd.Series(station_coords.get(station_name, (None, None)), index=['Latitude', 'Longitude'])

df_wind[['Latitude', 'Longitude']] = df_wind['Station Name'].apply(get_coords)


# Reproject everything to UTM for accurate spatial operations
to_utm = lambda gdf: gdf.to_crs(epsg=32616)
gdf_clusters_utm = to_utm(gdf_clusters)
gdf_bus_stops = to_utm(gdf_bus_stops)
gdf_urban_farms = to_utm(gdf_urban_farms)
gdf_rail_lines = to_utm(gdf_rail_lines)
gdf_alt_fuel = to_utm(gdf_alt_fuel)
gdf_census = gdf_census.to_crs("EPSG:4326")  # keep in WGS84 for join

# 1. Count of bus stops within 250m 
buffer = gdf_clusters_utm.copy()
buffer['geometry'] = buffer.geometry.buffer(250)
joined = gpd.sjoin(gdf_bus_stops, buffer, how="inner", predicate='within')
counts = joined.groupby('index_right').size()
gdf_clusters_utm['bus_stop_count'] = gdf_clusters_utm.index.map(counts).fillna(0).astype(int)

# 2. Distance to nearest urban farm 
gdf_clusters_utm['dist_urban_farm'] = gdf_clusters_utm.geometry.apply(
    lambda pt: gdf_urban_farms.distance(pt).min()
)

# 3. Distance to nearest rail line 
gdf_clusters_utm['dist_rail_line'] = gdf_clusters_utm.geometry.apply(
    lambda pt: gdf_rail_lines.distance(pt).min()
)

# 4. Distance to nearest alternative fuel station 
gdf_clusters_utm['dist_alt_fuel'] = gdf_clusters_utm.geometry.apply(
    lambda pt: gdf_alt_fuel.distance(pt).min()
)

# 5. Add Census vulnerability index (spatial join) 
# Convert cluster_centroids to GeoDataFrame
gdf_clusters = gpd.GeoDataFrame(
    cluster_centroids,
    geometry=[Point(xy) for xy in zip(cluster_centroids.longitude, cluster_centroids.latitude)],
    crs="EPSG:4326"
)

# Bring 'geohash' from features_df before reprojection
# This assumes features_df has a 'geohash' column corresponding to each cluster centroid
# You might need to adjust the merge logic if there's no direct correspondence
gdf_clusters = gdf_clusters.merge(features_df[['geohash', 'longitude', 'latitude']], on=['longitude', 'latitude'], how='left')


# Merge with features_df to get 'time_block' 
gdf_clusters = gdf_clusters.merge(
    features_df[['geohash', 'time_block']],
    left_on='geohash',
    right_on='geohash',
    how='left'
)

# the actual column name is 'Vulnerability_Score'
vulnerability_col_name = 'COMMAREA_N'
gdf_enriched = gpd.sjoin(
    gdf_clusters,
    gdf_census[['geometry', vulnerability_col_name]],
    how='left',
    predicate='within'
)
print(" External layers integrated with cluster data.")

In [ ]:
# Step 9: Visualize Sensor Locations with Folium
# Create map centered on Chicago
m = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

#  cluster_centroids is a DataFrame with your cluster data
# Convert cluster_centroids to a GeoDataFrame if it's not already
gdf_clusters = gpd.GeoDataFrame(
    cluster_centroids,
    geometry=[Point(xy) for xy in zip(cluster_centroids.longitude, cluster_centroids.latitude)],
    crs="EPSG:4326"  # Assuming your data is in WGS84
)

# Add sensor locations from your cluster GeoDataFrame
for _, row in gdf_clusters.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=f"Time Block: {row.get('time_block', 'N/A')}" #add a time block info
    ).add_to(m)

# Save and display
m.save("sensor_locations_map.html")

from google.colab import files
files.download("sensor_locations_map.html")

In [ ]:
# Cluster Visualization (DBSCAN & KMeans)

# Set seaborn style for better aesthetics in plots
sns.set(style="whitegrid", palette="muted", color_codes=True)

# Plot DBSCAN clustering results using longitude and latitude
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='longitude', y='latitude',
    hue='dbscan_cluster', data=features_df,
    palette='Set1', legend='full'
)
plt.title('DBSCAN Clusters')  # Title for the DBSCAN plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Position the legend outside the plot
plt.show()

# Plot KMeans clustering results using longitude and latitude
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='longitude', y='latitude',
    hue='kmeans_cluster', data=features_df,
    palette='Set2', legend='full'
)
plt.title('KMeans Clusters')  # Title for the KMeans plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Position the legend outside the plot
plt.show()


In [ ]:
# Define a list of eps values to test for DBSCAN clustering
eps_values = [0.02, 0.03, 0.04, 0.05, 0.06]

# Initialize a list to store silhouette scores for each eps
scores = []

# Iterate through each eps value
for eps in eps_values:
    # Create a DBSCAN model with current eps and fixed min_samples=5
    db = DBSCAN(eps=eps, min_samples=5)

    # Fit DBSCAN and get cluster labels
    labels = db.fit_predict(X_scaled)

    # Create a mask to exclude noise points (label = -1)
    mask = labels != -1

    # If there are enough non-noise points, calculate silhouette score
    if mask.sum() > 1:
        # Compute silhouette score for current clustering (excluding noise)
        score = silhouette_score(X_scaled[mask], labels[mask])
    else:
        # Assign a default low score if too few points to compute a valid score
        score = -1

    # Store the score in the list
    scores.append(score)

    # Print the current eps value and its corresponding silhouette score
    print(f"eps={eps} => Silhouette Score: {score:.4f}")
    

In [ ]:
# Tuning n_clusters for KMeans:
range_n = range(2, 11)  # Set the range of cluster numbers (from 2 to 10) to evaluate KMeans with different numbers of clusters
kmeans_scores = []  # Initialize an empty list to store silhouette scores for each k

for k in range_n:  # Iterate over the range of possible cluster numbers (2 to 10)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')  # Initialize KMeans with k clusters and fixed random state for reproducibility
    labels = kmeans.fit_predict(X_scaled)  # Fit the KMeans model to the scaled data and predict cluster labels
    score = silhouette_score(X_scaled, labels)  # Calculate the silhouette score to measure clustering quality
    kmeans_scores.append(score)  # Append the silhouette score to the list
    print(f"k={k} => Silhouette Score: {score:.4f}")  # Print the silhouette score for the current value of k


In [ ]:
# Plot Scores to Choose Best Parameters
# Create a new figure for plotting the DBSCAN Silhouette Scores
plt.figure(figsize=(8, 4))

# Plot the Silhouette Scores for different 'eps' values (DBSCAN) with markers at each point
plt.plot(eps_values, scores, marker='o', label='DBSCAN')

# Set the title of the plot to indicate it's showing Silhouette Score vs eps for DBSCAN
plt.title('Silhouette Score vs eps (DBSCAN)')

# Label the x-axis as 'eps' (epsilon value for DBSCAN)
plt.xlabel('eps')

# Label the y-axis as 'Silhouette Score'
plt.ylabel('Silhouette Score')

# Enable grid lines for better readability
plt.grid(True)

# Display the legend to indicate the line represents DBSCAN
plt.legend()

# Show the plot
plt.show()

# Create another figure for plotting the KMeans Silhouette Scores
plt.figure(figsize=(8, 4))

# Plot the Silhouette Scores for different 'n_clusters' values (KMeans) with markers at each point
plt.plot(list(range_n), kmeans_scores, marker='o', color='orange', label='KMeans')

# Set the title of the plot to indicate it's showing Silhouette Score vs n_clusters for KMeans
plt.title('Silhouette Score vs n_clusters (KMeans)')

# Label the x-axis as 'n_clusters' (number of clusters for KMeans)
plt.xlabel('n_clusters')

# Label the y-axis as 'Silhouette Score'
plt.ylabel('Silhouette Score')

# Enable grid lines for better readability
plt.grid(True)

# Display the legend to indicate the line represents KMeans
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Step-by-Step: Stratified Sampling by Geohash (Before vs After) 
# We'll assume you want to compare the full dataset vs a stratified sampled version — e.g., 30% of each geohash group.

# 1. Plot Full Dataset (Before Sampling)
plt.figure(figsize=(10, 6))  # Create a new figure with a defined size for better visibility.

# Plot a scatter plot showing data distribution by geohash
sns.scatterplot(
    data=features_df,          # Use the full features dataset
    x='longitude',             # Set x-axis to longitude
    y='latitude',              # Set y-axis to latitude
    hue='geohash',             # Color points by geohash to show geographic grouping
    legend=False,              # Hide the legend for clarity if there are many geohash groups
    s=20                       # Set marker size for better visualization
)

plt.title("Full Dataset: Geographical Distribution by Geohash")  # Add plot title
plt.xlabel("Longitude")  # Label x-axis
plt.ylabel("Latitude")   # Label y-axis
plt.show()               # Display the plot